In [1]:
pip install ultralyticsplus==0.0.23 ultralytics==8.0.21 pymupdf pdfplumber

  Using cached ultralyticsplus-0.0.23-py3-none-any.whl (11 kB)
ERROR: Ignored the following yanked versions: 8.0.129, 8.0.174, 8.0.177
ERROR: Ignored the following versions that require a different python version: 8.0.10 Requires-Python >=3.7,<=3.11; 8.0.11 Requires-Python >=3.7,<=3.11; 8.0.12 Requires-Python >=3.7,<=3.11; 8.0.13 Requires-Python >=3.7,<=3.11; 8.0.14 Requires-Python >=3.7,<=3.11; 8.0.15 Requires-Python >=3.7,<=3.11; 8.0.16 Requires-Python >=3.7,<=3.11; 8.0.17 Requires-Python >=3.7,<=3.11; 8.0.18 Requires-Python >=3.7,<=3.11; 8.0.19 Requires-Python >=3.7,<=3.11; 8.0.20 Requires-Python >=3.7,<=3.11; 8.0.21 Requires-Python >=3.7,<=3.11; 8.0.22 Requires-Python >=3.7,<=3.11; 8.0.23 Requires-Python >=3.7,<=3.11; 8.0.24 Requires-Python >=3.7,<=3.11; 8.0.25 Requires-Python >=3.7,<=3.11; 8.0.26 Requires-Python >=3.7,<=3.11; 8.0.27 Requires-Python >=3.7,<=3.11; 8.0.28 Requires-Python >=3.7,<=3.11; 8.0.29 Requires-Python >=3.7,<=3.11; 8.0.30 Requires-Python >=3.7,<=3.11; 8.0.31 Re

In [2]:
import fitz # imports the pymupdf library
import io

from PIL import Image
def get_pil_from_pdf_page(page: fitz.Page, dpi: int = 300) -> Image:
    return Image.open(io.BytesIO(page.get_pixmap(dpi=dpi).tobytes(output="png")))


doc = fitz.open("../fintabnet/pdf/AKAM/2005/page_75.pdf") # open a document
page = doc[0]

image = get_pil_from_pdf_page(page)

In [21]:
from ultralyticsplus import YOLO, render_result

# load model
model = YOLO('keremberke/yolov8s-table-extraction')

# set model parameters
model.overrides['conf'] = 0.25  # NMS confidence threshold
model.overrides['iou'] = 0.45  # NMS IoU threshold
model.overrides['agnostic_nms'] = False  # NMS class-agnostic
model.overrides['max_det'] = 1000  # maximum number of detections per image

# perform inference
results = model.predict(image)

# observe results
print(results)
render = render_result(model=model, image=image, result=results[0])
#render.show()
render.save("test.png")



Ultralytics YOLOv8.0.21 🚀 Python-3.9.18 torch-2.1.2+cu121 CPU
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs
0: 640x512 2 borderlesss, 111.8ms
Speed: 0.4ms pre-process, 111.8ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)


[Ultralytics YOLO <class 'ultralytics.yolo.engine.results.Boxes'> masks
type: <class 'torch.Tensor'>
shape: torch.Size([2, 6])
dtype: torch.float32
 + tensor([[3.61000e+02, 1.74400e+03, 2.10500e+03, 2.89200e+03, 9.48168e-01, 1.00000e+00],
        [3.66000e+02, 6.07000e+02, 2.10600e+03, 1.31500e+03, 8.92498e-01, 1.00000e+00]])]


In [22]:
import pdfplumber
with pdfplumber.open("../fintabnet/pdf/AKAM/2005/page_75.pdf") as pdf:
    page = pdf.pages[0]
    b = results[0].boxes[0]
    bbox = b.xyxy.tolist()[0]

    im = page.to_image(resolution=300)
    im.draw_rect([x/im.scale for x in bbox])
    # boxes vector is bounding box but only first two values the last values stands presumably for the borderless variable an the other one for the value behind it
    im.save("test.png")